In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from time import sleep
from bs4 import BeautifulSoup
import lxml.html as lh
from datetime import datetime
from lxml import etree

options = webdriver.ChromeOptions() 
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36'})
print(driver.execute_script("return navigator.userAgent;"))
driver.get('https://www.httpbin.org/headers')

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36


In [3]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)     # Display all rows
np.set_printoptions(threshold=np.inf)       # Display entire numpy arrays

In [4]:
url = f'https://fbref.com/en/comps/Big5/possession/squads/Big-5-European-Leagues-Stats'
driver.get(url)
driver.implicitly_wait(10)

html_content = driver.page_source

# Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find all table rows in the tbody section
table = soup.find('table', id= f"stats_teams_possession_for")
# then we can iterate through each row and extract either header or row values:
header = []
rows = []
for j, row in enumerate(table.find_all('tr')):
    if j < 2:
        header = [el.text.strip() for el in row.find_all('th')]
    else:
        rows.append([el.text.strip() for el in row.find_all('td')])
header.pop(0)

data = pd.DataFrame(data=rows, columns=header)
data.head()

,Squad,Comp,# Pl,Poss,90s,Touches,Def Pen,Def 3rd,Mid 3rd,Att 3rd,Att Pen,Live,Att,Succ,Succ%,Tkld,Tkld%,Carries,TotDist,PrgDist,PrgC,1/3,CPA,Mis,Dis,Rec,PrgR
0,Ajaccio,fr Ligue 1,36,43.2,38.0,19271,1979,6134,9380,3966,486,19262,630,283,44.9,277,44.0,12563,73525,32335,487,406,92,625,324,11114,1150
1,Almería,es La Liga,29,45.1,38.0,20070,2715,7746,8468,4071,631,20067,666,296,44.4,310,46.5,10790,56980,29134,569,432,131,580,281,12047,1080
2,Angers,fr Ligue 1,33,46.9,38.0,21515,2041,6588,10633,4609,567,21511,928,441,47.5,392,42.2,15521,89958,41416,632,596,149,593,349,13974,1374
3,Arsenal,eng Premier League,26,59.3,38.0,25909,1994,6632,11452,8015,1296,25905,751,351,46.7,290,38.6,15923,84011,44201,824,583,281,526,378,18101,2024
4,Aston Villa,eng Premier League,26,49.3,38.0,21501,2783,7912,9077,4706,746,21496,681,292,42.9,279,41.0,12294,65583,33156,637,434,174,565,371,13587,1227


In [6]:
data.to_csv('SquadPossessionStats.csv', index=False)